In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

## Model evaluators
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from sklearn.metrics import plot_roc_curve # note: this was changed in Scikit-Learn 1.2+ to be "RocCurveDisplay" (see below)
from sklearn.metrics import RocCurveDisplay # new in Scikit-Learn 1.2+

In [74]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 174.8 kB/s eta 0:00:06
   -------------- ------------------------- 0.5/1.5 MB 174.8 kB/s eta 0:00:06
   -------------- ------------------------- 0.5/1.5 MB 174.8 kB/s eta 0:00:06
   -------------- ------------------------- 0.5/1.5 MB 174.8 kB/s eta 0:00:06
   -------------- ------------------------- 0.5/1.5 MB 174.8 kB/s e

In [80]:
%%time
!pip install catboost

CPU times: total: 15.6 ms
Wall time: 1.71 s


In [54]:
train = pd.read_csv(r"C:\Users\navad\Desktop\3.Space_titanic\train.csv")
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [55]:
train.shape

(8693, 14)

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [57]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [58]:
#converting object types to categorical types
for label,content in train.items():
    if pd.api.types.is_object_dtype(content):
        train[label] = content.astype("category").cat.as_ordered()

In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   PassengerId   8693 non-null   category
 1   HomePlanet    8492 non-null   category
 2   CryoSleep     8476 non-null   category
 3   Cabin         8494 non-null   category
 4   Destination   8511 non-null   category
 5   Age           8514 non-null   float64 
 6   VIP           8490 non-null   category
 7   RoomService   8512 non-null   float64 
 8   FoodCourt     8510 non-null   float64 
 9   ShoppingMall  8485 non-null   float64 
 10  Spa           8510 non-null   float64 
 11  VRDeck        8505 non-null   float64 
 12  Name          8493 non-null   category
 13  Transported   8693 non-null   bool    
dtypes: bool(1), category(7), float64(6)
memory usage: 1.4 MB


In [60]:
#converting all the categorical values into numbers and filling the data.
for label,content in train.items():
    if not pd.api.types.is_numeric_dtype(content):
        train[label] = content.astype("category").cat.codes
        if pd.isna(content).sum():
            train[label] = pd.Categorical(content).codes + 1

In [61]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   int16  
 1   HomePlanet    8693 non-null   int8   
 2   CryoSleep     8693 non-null   int8   
 3   Cabin         8693 non-null   int16  
 4   Destination   8693 non-null   int8   
 5   Age           8514 non-null   float64
 6   VIP           8693 non-null   int8   
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8693 non-null   int16  
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), int16(3), int8(4)
memory usage: 501.0 KB


In [62]:
L = list(pd.Categorical(train["CryoSleep"]).codes)
for i in range(len(L)):
    if L[i] == -1:
        print(L(i))#just checking a sample column if it has any values assigned as -1

In [63]:
X = train.drop("Transported",axis = 1)
Y = train["Transported"]
X.shape,Y.shape

((8693, 13), (8693,))

In [64]:
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid = train_test_split(X,Y,test_size = 0.2)
X_train.shape,Y_train.shape,X_valid.shape,Y_valid.shape

((6954, 13), (6954,), (1739, 13), (1739,))

In [65]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6954 entries, 2146 to 2025
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6954 non-null   int16  
 1   HomePlanet    6954 non-null   int8   
 2   CryoSleep     6954 non-null   int8   
 3   Cabin         6954 non-null   int16  
 4   Destination   6954 non-null   int8   
 5   Age           6812 non-null   float64
 6   VIP           6954 non-null   int8   
 7   RoomService   6817 non-null   float64
 8   FoodCourt     6814 non-null   float64
 9   ShoppingMall  6788 non-null   float64
 10  Spa           6809 non-null   float64
 11  VRDeck        6811 non-null   float64
 12  Name          6954 non-null   int16  
dtypes: float64(6), int16(3), int8(4)
memory usage: 448.2 KB


In [66]:
for label,content in X_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            X_train[label] = content.fillna(content.median())

In [67]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6954 entries, 2146 to 2025
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6954 non-null   int16  
 1   HomePlanet    6954 non-null   int8   
 2   CryoSleep     6954 non-null   int8   
 3   Cabin         6954 non-null   int16  
 4   Destination   6954 non-null   int8   
 5   Age           6954 non-null   float64
 6   VIP           6954 non-null   int8   
 7   RoomService   6954 non-null   float64
 8   FoodCourt     6954 non-null   float64
 9   ShoppingMall  6954 non-null   float64
 10  Spa           6954 non-null   float64
 11  VRDeck        6954 non-null   float64
 12  Name          6954 non-null   int16  
dtypes: float64(6), int16(3), int8(4)
memory usage: 448.2 KB


In [68]:
for label,content in X_valid.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            X_valid[label] = content.fillna(content.median())

In [69]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1739 entries, 1195 to 3694
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   1739 non-null   int16  
 1   HomePlanet    1739 non-null   int8   
 2   CryoSleep     1739 non-null   int8   
 3   Cabin         1739 non-null   int16  
 4   Destination   1739 non-null   int8   
 5   Age           1739 non-null   float64
 6   VIP           1739 non-null   int8   
 7   RoomService   1739 non-null   float64
 8   FoodCourt     1739 non-null   float64
 9   ShoppingMall  1739 non-null   float64
 10  Spa           1739 non-null   float64
 11  VRDeck        1739 non-null   float64
 12  Name          1739 non-null   int16  
dtypes: float64(6), int16(3), int8(4)
memory usage: 112.1 KB


Now the data is ready with all the data in numbers and no missing values. We can fit the models!

In [87]:
#make a fucntion of models to fit the data
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=124),
    "KNieighbour": KNeighborsClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.05, use_label_encoder=False, eval_metric="logloss", random_state=42),
    "LightGBM": LGBMClassifier(n_estimators=100, random_state=42),
    "CatBoost": CatBoostClassifier(n_estimators=100, verbose=0, random_state=42)
}

In [88]:
import time
results = {}
for name,model in models.items():
    start_time = time.time()
    
    #fit each of the model
    model.fit(X_train,Y_train)

    #making predictions
    Y_preds = model.predict(X_valid)

    acc = accuracy_score(Y_valid,Y_preds)

    results[name] = acc

    print(f"{name}: Accuracy = {acc:.4f} | Time taken = {time.time() - start_time:.2f} sec")
    

C:\Users\navad\Desktop\env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Accuracy = 0.7878 | Time taken = 1.40 sec
Random Forest: Accuracy = 0.7890 | Time taken = 2.88 sec
KNieighbour: Accuracy = 0.7303 | Time taken = 0.46 sec
Gradient Boosting: Accuracy = 0.8062 | Time taken = 3.62 sec


C:\Users\navad\Desktop\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:08:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost: Accuracy = 0.8045 | Time taken = 2.56 sec
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2133
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230
LightGBM: Accuracy = 0.8079 | Time taken = 0.65 sec
CatBoost: Accuracy = 0.8056 | Time taken = 1.05 sec


In [89]:
# Convert results to a DataFrame
results_df = pd.DataFrame.from_dict(results, orient="index", columns=["Accuracy"])
results_df = results_df.sort_values(by="Accuracy", ascending=False)

# Display results
print(results_df)

                     Accuracy
LightGBM             0.807936
Gradient Boosting    0.806210
CatBoost             0.805635
XGBoost              0.804485
Random Forest        0.788959
Logistic Regression  0.787809
KNieighbour          0.730305


In [90]:
model_ = LGBMClassifier(n_estimators=100, random_state=42)
model_.fit(X_train,Y_train)


[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2133
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


LGBMClassifier(random_state=42)

In [131]:
test = pd.read_csv(r"C:\Users\navad\Desktop\3.Space_titanic\test.csv")
test_temp = pd.read_csv(r"C:\Users\navad\Desktop\3.Space_titanic\test.csv") 
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [123]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [124]:
test.shape

(4277, 13)

In [125]:
for label,content in test.items():
    if pd.api.types.is_object_dtype(content):
        #test[label] = content.astype("category").cat.as_ordered()
        test[label] = pd.Categorical(content).codes
        
for label,content in test.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            test[label] = pd.Categorical(content).codes + 1 
        

In [126]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   int16  
 1   HomePlanet    4277 non-null   int8   
 2   CryoSleep     4277 non-null   int8   
 3   Cabin         4277 non-null   int16  
 4   Destination   4277 non-null   int8   
 5   Age           4186 non-null   float64
 6   VIP           4277 non-null   int8   
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4277 non-null   int16  
dtypes: float64(6), int16(3), int8(4)
memory usage: 242.4 KB


In [127]:
for label,content in test.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            test[label] = content.fillna(content.median()) 

In [128]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   int16  
 1   HomePlanet    4277 non-null   int8   
 2   CryoSleep     4277 non-null   int8   
 3   Cabin         4277 non-null   int16  
 4   Destination   4277 non-null   int8   
 5   Age           4277 non-null   float64
 6   VIP           4277 non-null   int8   
 7   RoomService   4277 non-null   float64
 8   FoodCourt     4277 non-null   float64
 9   ShoppingMall  4277 non-null   float64
 10  Spa           4277 non-null   float64
 11  VRDeck        4277 non-null   float64
 12  Name          4277 non-null   int16  
dtypes: float64(6), int16(3), int8(4)
memory usage: 242.4 KB


In [107]:
Y_preds = model_.predict(test)
Y_preds

array([False, False,  True, ...,  True,  True,  True])

In [115]:
output = pd.DataFrame({"PassengerId" : test_temp["PassengerId"]  , "Transported" : Y_preds}  )
output

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [117]:
output.to_csv("Spacetitanic_Submission_1.csv",index = False)

Now lets create another predictions with training of all data

In [118]:
X_trainandvalid = pd.concat([X_train,X_valid],ignore_index=True)
Y_trainandvalid = pd.concat([Y_train,Y_valid],ignore_index=True)
X_trainandvalid.shape,Y_trainandvalid.shape

((8693, 13), (8693,))

In [119]:
model_full = LGBMClassifier(n_estimators=100, random_state=42)
model_full.fit(X_trainandvalid,Y_trainandvalid)


[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001090 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2133
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495


LGBMClassifier(random_state=42)

In [121]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [129]:
Y_predsfull = model_full.predict(test)
Y_predsfull

array([False, False,  True, ...,  True,  True,  True])

In [132]:
output2 = pd.DataFrame({"PassengerId" : test_temp["PassengerId"]  , "Transported" : Y_predsfull}  )
output2.to_csv("Spacetitanic_Submission_2.csv",index = False)